In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.8e}'.format

import configparser
from collections import defaultdict
from src.character import Character, CharacterType 
from src.stats import random_stats
from src.selection import stochastic_tournament_selection, deterministic_tournament_selection, add_relative_accumulate, elitist_selection, roulette_wheel_selection, universal_selection, boltzmann_selection, rank_based_selection
from src.genes import  encode_genes, decode_genes
from src.mutation import one_gene_mutation, multi_gene_mutation, multi_gene_mutation_uniform

In [2]:
import sys
sys.path.append("src")

from src.population import generate_init_population, eval_performace
from src.crossover import single_point_crossover, two_point_crossover, uniform_crossover, annular_crossover, normalize_chromosome


In [3]:
def read_config(filename):
    config = configparser.ConfigParser()
    config.read(filename)
    params = defaultdict(dict)
    for section in config.sections():
        for key, value in config.items(section):
            params[section][key] = value
    return params

params = read_config('config_file.config')
params

defaultdict(dict,
            {'CHARACTER': {'type': 'CharacterType.ARCHER'},
             'N': {'n': '200'},
             'M': {'m': '150'},
             'K': {'k': '120'},
             'GENERATIONS': {'number': '500'},
             'METHOD1PERCENTAGE': {'percentage': '0.4'},
             'SELECTION_METHOD': {'method1': "'elitist'",
              'method2': "'roulette_wheel'"},
             'BOLTZMANN': {'t_0': '50',
              't_offset': '10',
              'delta_t': '0.015',
              'm': '0.01'},
             'deterministic_tournament_selection': {'tournament_size': '50'},
             'stochastic_tournament_selection': {'threshold': '0.6'},
             'metodo_cruza': {'crossover': "'one_point'"},
             'MUTATION': {'mutation_type': "'multi_gene'",
              'mutation_rate': '0.7'},
             'METHODRePERCENTAGE': {'percentage': '0.6'},
             'metodo_reemplazo': {'metodo1': "'rank_based'",
              'metodo2': "'deterministic_tournament'"},
    

In [4]:
from collections import namedtuple

def parameters():
    
    config_params = read_config('config_file.config')

    # Personaje 
    type = eval(config_params['CHARACTER']['type'])
    
    # sum( stats ) = 150
    maxStatsValue = eval(config_params['M']['m'])
    
    # N
    populationNumber = eval(config_params['N']['n'])
    # K
    k = eval(config_params['K']['k'])

    # Numero máximo de generaciones - criterio de corte
    numberOfGenerations = eval(config_params['GENERATIONS']['number'])
    
    # A%
    method1Percentage = eval(config_params['METHOD1PERCENTAGE']['percentage'])
    selectionMethod1 = eval(config_params['SELECTION_METHOD']['method1'])
    selectionMethod2 = eval(config_params['SELECTION_METHOD']['method2'])
   
    # CrossOver
    metodo_cruza = eval(config_params['metodo_cruza']['crossover'])
 
    # Mutation
    mutation_type = eval(config_params['MUTATION']['mutation_type'])
    mutation_rate = eval(config_params['MUTATION']['mutation_rate'])

    # B%
    methodReplacePercentage = eval(config_params['METHODRePERCENTAGE']['percentage'])
    metodo_reemplazo3 = eval(config_params['metodo_reemplazo']['metodo1'])
    metodo_reemplazo4 = eval(config_params['metodo_reemplazo']['metodo2'])

    cutCriteria = eval(config_params['CUT_CRITERIA']['criteria_type'])
    
    # Selector parameters
    # M
    tournament_size = eval(config_params['deterministic_tournament_selection']['tournament_size'])
    # stochastic_tournament_selection
    threhold = eval(config_params['stochastic_tournament_selection']['threshold'])
    # Boltzmann parameters
    boltzmannT_0 = eval(config_params['BOLTZMANN']['t_0'])
    boltzmannT_c = eval(config_params['BOLTZMANN']['t_offset'])
    boltzmannM = eval(config_params['BOLTZMANN']['m'])

    Param = namedtuple('Param',[
        'character',
        'maxStatsValue',
        'populationNumber',
        'k',
        'numberOfGenerations',
        'method1Percentage',
        'selectionMethod1',
        'selectionMethod2',
        'metodo_cruza',
        'mutation_type',
        'mutation_rate',
        'methodReplacePercentage',
        'metodo_reemplazo3',
        'metodo_reemplazo4',
        'cutCriteria',
        'tournament_size',
        'threhold',
        'boltzmannT_0',
        'boltzmannT_c',
        'boltzmannM'
    ])    

    parametros = Param(
                character=type,
                maxStatsValue=maxStatsValue,
                populationNumber=populationNumber,
                k=k,
                numberOfGenerations=numberOfGenerations,
                method1Percentage=method1Percentage,
                selectionMethod1=selectionMethod1,
                selectionMethod2=selectionMethod2,
                metodo_cruza=metodo_cruza,
                mutation_type=mutation_type,
                mutation_rate=mutation_rate,
                methodReplacePercentage=methodReplacePercentage,
                metodo_reemplazo3=metodo_reemplazo3,
                metodo_reemplazo4=metodo_reemplazo4,
                cutCriteria=cutCriteria,
                tournament_size=tournament_size,
                threhold=threhold,
                boltzmannT_0=boltzmannT_0,
                boltzmannT_c=boltzmannT_c,
                boltzmannM=boltzmannM
    )

    return parametros

p = parameters()


In [5]:
from src.character import Character, CharacterType 
import sys
sys.path.append("src")

from src.population import generate_init_population

#GENERACION 0

generation_0 = generate_init_population(p.populationNumber,p.maxStatsValue,p.character)
generacion = 0


In [6]:
# GUARDO EL MAS APTO DE LA GENERACION 0
# Escribe la poblacion inicial.
#generation_0.insert(0, 'generation', int(generacion))
generation_0['generation'] = generacion
mas_apto_G0 = generation_0.head(1)
mas_apto_G0.to_csv(f'datos-Prueba1.csv', mode='w', index=False)


In [7]:

def method_selector(Kt, method1Percentage): 
    # Selecciono K individuos
    # Utilizo el porcentaje para cada método
    K1 = int(np.round(Kt * method1Percentage))
    K2 = Kt - K1
    return K1, K2

k1, k2 = method_selector(p.k, p.method1Percentage)

In [8]:
current_generation = generation_0
generacion = 0

In [9]:
def selection_method(poblacion, ki, params, method, generation):
    
    print(f' Generacion: {generation}')
    poblacion_rel_acu = add_relative_accumulate(poblacion)

    if(method == 'elitist'):
        selected = elitist_selection(poblacion_rel_acu, ki)
    elif(method == 'roulette_wheel'):
        selected = roulette_wheel_selection(poblacion_rel_acu, ki)
    elif(method == 'boltzmann'):
        selected = boltzmann_selection(poblacion_rel_acu, ki, generation, params.boltzmannT_0,params.boltzmannT_c,params.boltzmannM)
        #print(newPopulationRoulette)
    elif(method == 'universal_selection'):
        selected = universal_selection(poblacion_rel_acu, ki)
        #print(newPopulationRoulette)
    elif(method == 'deterministic_tournament'):
        selected = deterministic_tournament_selection(poblacion_rel_acu, params.tournament_size, ki)
        #print(newPopulationRoulette)
    elif(method == 'stochastic_tournament'):
        selected = stochastic_tournament_selection(poblacion_rel_acu, ki, params.threshold)
        #print(newPopulationRoulette)
    elif(method == 'rank_based'):
        selected = rank_based_selection(poblacion_rel_acu, ki)
        #print(newPopulationRoulette)
    return selected

# SELECCION DE PADRES
generacion_sel1 = selection_method(current_generation, k1, p, p.selectionMethod1, generacion)
generacion_sel2 = selection_method(current_generation, k2, p, p.selectionMethod2, generacion)

# SELECCION TOTAL: K padres
selection_total = pd.concat([generacion_sel1, generacion_sel2])

selection_total.drop(columns=['performance_relative'], inplace=True)
selection_total.drop(columns=['performance_accumulated'], inplace=True)
selection_total = selection_total.sort_values(by='performance', ascending=False)
selection_total.reset_index(drop=True, inplace=True)


 Generacion: 0
 Generacion: 0


In [10]:
cromosomas = encode_genes(selection_total)

cromosomas

,strength,agility,expertise,resistance,life,height
1,00111011,00000101,01000011,00001001,00001010,001
2,01000001,00000100,00111110,00010001,00000010,100
3,01000001,00000100,00111110,00010001,00000010,100
4,01010100,00000011,00101001,00001000,00001110,011
5,01010100,00000011,00101001,00001000,00001110,011
...,...,...,...,...,...,...
116,00011101,00111110,00000010,00010001,00101000,011
117,00011111,00010000,00000111,01010011,00001101,111
118,00010101,00110111,00000011,00101111,00011000,011
119,00010100,00000110,00001001,01010100,00011111,010


In [11]:
def crossover_method(poblacion, method):
    
    if(method == 'one_point'):
        crossed = single_point_crossover(poblacion)
    elif(method == 'two_point'):
        crossed = two_point_crossover(poblacion)
    elif(method == 'uniform'):
        crossed = uniform_crossover(poblacion)
    elif(method == 'annular'):
        crossed = annular_crossover(poblacion)
    
    return crossed

crossed_result = crossover_method(cromosomas, p.metodo_cruza)

crossed_result

,strength,agility,expertise,resistance,life,height
0,01000001,00000100,01111110,00010001,00001010,100
1,00111011,00000101,00000011,00001001,00000010,001
2,01010100,00000011,00111001,00001000,00000010,111
3,01000001,00000100,00101110,00010001,00001110,000
4,01010100,00000011,00101001,00001000,00001110,011
...,...,...,...,...,...,...
115,00011111,00111111,00100110,00000010,00101000,011
116,00011101,00110111,00000011,01010011,00011000,011
117,00010111,00010000,00000111,00101111,00001101,111
118,00001001,00100010,00000110,00010000,01010101,100


In [12]:
def mutation_method(poblacion, rate, method):
    
    if(method == 'one_gene'):
        mutated = one_gene_mutation(poblacion, rate)
    elif(method == 'multi_gene'):
        mutated = multi_gene_mutation(poblacion, rate)
    elif(method == 'multi_gene_mutation_uniform'):
        mutated = multi_gene_mutation_uniform(poblacion, rate)
    
    return mutated

mutation_result = mutation_method(crossed_result, p.mutation_rate, p.mutation_type)

mutation_result

     strength   agility expertise resistance      life height
0    01000001  00000100  01111110   00010001  00001010    100
1    00111011  00000101  00000011   00001001  00000010    001
2    01010100  00000011  00111001   00001000  00000010    111
3    01000001  00000100  00101110   00010001  00001110    000
4    01010100  00000011  00101001   00001000  00001110    011
..        ...       ...       ...        ...       ...    ...
115  00011111  00111111  00100110   00000010  00101000    011
116  00011101  00110111  00000011   01010011  00011000    011
117  00010111  00010000  00000111   00101111  00001101    111
118  00001001  00100010  00000110   00010000  01010101    100
119  00010100  00000110  00001001   01010100  00011111    010

[120 rows x 6 columns]


,strength,agility,expertise,resistance,life,height
0,11011110,00000100,10100011,10101101,10110101,100
1,00110100,00011010,00000011,01110110,00000010,010
2,01010100,10111100,10001110,11111101,11111111,100
3,10011010,01001110,11000100,00010001,00001110,000
4,00100100,11111100,00101001,00111111,11111011,001
...,...,...,...,...,...,...
115,00011111,11010001,00100110,11111101,00101000,101
116,11000110,00010011,00000011,10100100,10100110,100
117,01101001,10101011,00111100,00101111,10111010,000
118,10110100,00100010,01101111,01111111,11000000,011


In [13]:
cromosomas_norm = normalize_chromosome(mutation_result)

cromosomas_norm

,strength,agility,expertise,resistance,life,height
1,00101100,00000000,00100000,00100010,00100100,100
2,00100110,00010011,00000010,01011000,00000001,010
3,00001101,00011110,00010111,00101001,00101001,100
4,00110010,00011001,01000000,00000101,00000100,000
5,00001000,00111010,00001001,00001110,00111010,001
...,...,...,...,...,...,...
116,00001000,00110110,00001001,01000010,00001010,101
117,00110101,00000101,00000000,00101100,00101101,100
118,00011011,00101101,00001111,00001100,00110001,000
119,00101001,00000111,00011001,00011101,00101100,011


In [14]:
cromosomas_norm_decode = decode_genes(cromosomas_norm)
offspring = eval_performace(cromosomas_norm_decode,p.character)

offspring = offspring.sort_values(by='performance', ascending=False)
offspring.reset_index(drop=True, inplace=True)

offspring

,strength,agility,expertise,resistance,life,height,characterType,performance
0,7.20000000e+01,0.00000000e+00,6.50000000e+01,0.00000000e+00,1.00000000e+01,1.80000000e+00,CharacterType.ARCHER,1.15249821e+03
1,4.40000000e+01,2.70000000e+01,6.20000000e+01,1.30000000e+01,2.00000000e+00,1.90000000e+00,CharacterType.ARCHER,8.24631729e+02
2,5.00000000e+01,2.50000000e+01,6.40000000e+01,5.00000000e+00,4.00000000e+00,1.30000000e+00,CharacterType.ARCHER,8.17813092e+02
3,4.80000000e+01,3.00000000e+01,6.20000000e+01,0.00000000e+00,9.00000000e+00,1.30000000e+00,CharacterType.ARCHER,7.84500218e+02
4,6.40000000e+01,9.00000000e+00,3.70000000e+01,2.10000000e+01,1.70000000e+01,1.90000000e+00,CharacterType.ARCHER,7.00920658e+02
...,...,...,...,...,...,...,...,...
115,5.00000000e+00,2.00000000e+00,1.30000000e+01,6.50000000e+01,6.20000000e+01,1.50000000e+00,CharacterType.ARCHER,2.91300814e+01
116,6.00000000e+00,2.30000000e+01,7.00000000e+00,6.20000000e+01,5.00000000e+01,1.80000000e+00,CharacterType.ARCHER,2.53341318e+01
117,5.30000000e+01,5.00000000e+00,0.00000000e+00,4.40000000e+01,4.50000000e+01,1.70000000e+00,CharacterType.ARCHER,1.47782845e+01
118,5.00000000e+00,1.20000000e+01,4.00000000e+00,6.20000000e+01,6.50000000e+01,1.70000000e+00,CharacterType.ARCHER,1.46221051e+01


In [15]:
# REEMPLAZO
# padres:  selection_total de tamaño K
# hijos: offspring  de tamaño K
# tomo K de los hijos
k_reemplazo_metodo3 = p.k
# el resto para completar N
k_reemplazo_metodo4 = p.populationNumber - p.k

# SELECCION DE PADRES
generacion_reemp1 = selection_method(offspring, k_reemplazo_metodo3, p, p.metodo_reemplazo3, generacion)
generacion_reemp2 = selection_method(selection_total, k_reemplazo_metodo4, p, p.metodo_reemplazo4, generacion)

# SELECCION TOTAL: K padres
new_generation = pd.concat([generacion_reemp1, generacion_reemp2])

new_generation.drop(columns=['performance_relative'], inplace=True)
new_generation.drop(columns=['performance_accumulated'], inplace=True)
new_generation = new_generation.sort_values(by='performance', ascending=False)
new_generation.reset_index(drop=True, inplace=True)

new_generation

 Generacion: 0
 Generacion: 0


,strength,agility,expertise,resistance,life,height,characterType,performance,generation
0,59,5,67,9,10,1.40000000e+00,CharacterType.ARCHER,1.01064621e+03,0
1,59,5,67,9,10,1.40000000e+00,CharacterType.ARCHER,1.01064621e+03,0
2,59,5,67,9,10,1.40000000e+00,CharacterType.ARCHER,1.01064621e+03,0
3,59,5,67,9,10,1.40000000e+00,CharacterType.ARCHER,1.01064621e+03,0
4,59,5,67,9,10,1.40000000e+00,CharacterType.ARCHER,1.01064621e+03,0
...,...,...,...,...,...,...,...,...,...
211,2.70000000e+01,4.80000000e+01,4.30000000e+01,8.00000000e+00,2.20000000e+01,1.50000000e+00,CharacterType.ARCHER,2.08333333e-01,NaN
212,3.10000000e+01,1.60000000e+01,4.40000000e+01,5.40000000e+01,3.00000000e+00,1.70000000e+00,CharacterType.ARCHER,1.75000000e-01,NaN
213,3.70000000e+01,3.60000000e+01,3.80000000e+01,8.00000000e+00,2.80000000e+01,1.60000000e+00,CharacterType.ARCHER,1.41666667e-01,NaN
214,4.20000000e+01,3.00000000e+00,4.40000000e+01,4.80000000e+01,1.00000000e+01,2.00000000e+00,CharacterType.ARCHER,1.00000000e-01,NaN


In [16]:
generacion += 1
# GUARDO EL MAS APTO DE LA GENERACION NUEVA
# Escribe la poblacion inicial.

#new_generation.insert(0, 'generation', int(generacion))
new_generation['generation'] = generacion
mas_apto_NG = new_generation.head(1)
mas_apto_NG.to_csv(f'datos-Prueba1.csv', mode='a', header=False, index=False)


ValueError: cannot insert generation, already exists